In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
database_type = "/full"

The `intakeOutput` table provides Intake and output recorded for patients entered from the nursing flowsheet.
See: http://eicu-crd.mit.edu/eicutables/intakeOutput/

Urine output is measured in mL/kg.
Urine output in mL/ Patient weight in kg.

#### Reading in lab dataset and keeping relevant columns to our study .

In [4]:
columns = ['patientunitstayid', 'intakeoutputoffset', 'intaketotal', 'outputtotal', 'celllabel']
intakeOutput = pd.read_csv('../../eICU' + database_type + '/intakeOutput.csv', usecols=columns)
intakeOutput

,patientunitstayid,intakeoutputoffset,intaketotal,outputtotal,celllabel
0,205505,661,0.00,0.0,Bodyweight (lb)
1,205505,661,0.00,0.0,Bodyweight (kg)
2,150708,3523,727.00,0.0,P.O.
3,150708,3523,727.00,0.0,Volume (mL)-sodium chloride 0.9 % flush IVPB 5...
4,158159,24801,120.00,0.0,P.O.
...,...,...,...,...,...
12030284,3335854,1140,152.10,0.0,Crystalloids
12030285,3334548,766,340.00,350.0,Crystalloids
12030286,3330152,5102,300.00,0.0,Nutrition Total
12030287,3327194,3834,101.18,55.0,Crystalloids


#### Reading in cleaned patient data

In [5]:
patient = pd.read_csv('../../eICU/training/patient.csv')
patient

,patientunitstayid,admissionweight
0,141168,84.3
1,141178,54.4
2,141179,176.4
3,141194,73.9
4,141196,194.7
...,...,...
200854,3353235,90.0
200855,3353237,78.4
200856,3353251,102.0
200857,3353254,83.9


#### Filtering out reading pertaining to urine output 

In [6]:
intakeOutputUrine = intakeOutput.loc[(intakeOutput['celllabel'].str.contains("Urine"))]
intakeOutputUrine

,patientunitstayid,intakeoutputoffset,intaketotal,outputtotal,celllabel
10,178962,-1507,240.0,400.0,Urine
29,162156,129,100.0,0.0,Urine
33,184045,9859,0.0,300.0,Urine
34,202558,-891,0.0,100.0,Urine
47,237097,5850,240.0,250.0,Urine
...,...,...,...,...,...
12030268,3339291,3337,0.0,400.0,Urine
12030273,3326946,1501,0.0,400.0,Urine
12030278,3336512,1710,100.0,230.0,Urine
12030279,3347200,12668,0.0,750.0,Urine


#### Merging on patient table to get patient weight

In [7]:
intakeOutputUrine = pd.DataFrame.merge(patient, intakeOutputUrine, on='patientunitstayid')

#### Calculating patient urine output by diving urine output in mL by the patient weight

In [8]:
intakeOutputUrine['urineoutputbyweight'] = intakeOutputUrine.apply(lambda x: x['outputtotal'] / x['admissionweight'], axis=1)

#### Dropping irrelevant columns to study.

In [9]:
intakeOutputUrine = intakeOutputUrine[['patientunitstayid','intakeoutputoffset', 'urineoutputbyweight']]
intakeOutputUrine

,patientunitstayid,intakeoutputoffset,urineoutputbyweight
0,141179,1420,3.968254
1,141179,933,2.834467
2,141179,1703,5.668934
3,141179,678,3.968254
4,141194,12201,1.353180
...,...,...,...
3331676,3353254,4169,19.070322
3331677,3353254,5326,4.767580
3331678,3353254,2626,10.131108
3331679,3353254,3101,15.494636


#### Saving the intakeOutputUrine DataFrame to a `csv` file

In [10]:
intakeOutputUrine

,patientunitstayid,intakeoutputoffset,urineoutputbyweight
0,141179,1420,3.968254
1,141179,933,2.834467
2,141179,1703,5.668934
3,141179,678,3.968254
4,141194,12201,1.353180
...,...,...,...
3331676,3353254,4169,19.070322
3331677,3353254,5326,4.767580
3331678,3353254,2626,10.131108
3331679,3353254,3101,15.494636


In [11]:
path = "../../eICU/training/"
intakeOutputUrine.to_csv(path + "intakeOutputUrine.csv", sep=',', index=False, encoding='utf-8')